In [35]:
!wget https://raw.githubusercontent.com/BunnyNoBugs/hse-headlines-project/main/data/test.csv
!wget https://raw.githubusercontent.com/BunnyNoBugs/hse-headlines-project/main/data/train.csv

--2023-03-23 16:31:10--  https://raw.githubusercontent.com/BunnyNoBugs/hse-headlines-project/main/data/test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19026 (19K) [text/plain]
Saving to: ‘test.csv.1’

test.csv.1          100%[===================>]  18.58K  --.-KB/s    in 0.001s  

2023-03-23 16:31:10 (20.1 MB/s) - ‘test.csv.1’ saved [19026/19026]

--2023-03-23 16:31:10--  https://raw.githubusercontent.com/BunnyNoBugs/hse-headlines-project/main/data/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 175396 (171K) [t

In [36]:
!wget https://raw.githubusercontent.com/BunnyNoBugs/hse-headlines-project/main/data/CorpusOfNewsHeadlines_new-Rus_FINAL.csv

--2023-03-23 16:31:10--  https://raw.githubusercontent.com/BunnyNoBugs/hse-headlines-project/main/data/CorpusOfNewsHeadlines_new-Rus_FINAL.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 980205 (957K) [text/plain]
Saving to: ‘CorpusOfNewsHeadlines_new-Rus_FINAL.csv.1’

CorpusOfNewsHeadlin 100%[===================>] 957.23K  --.-KB/s    in 0.04s   

2023-03-23 16:31:11 (22.6 MB/s) - ‘CorpusOfNewsHeadlines_new-Rus_FINAL.csv.1’ saved [980205/980205]



# TF-IDF

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import pandas as pd

In [44]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [45]:
train

,Headline,Type,Manipulative Device
0,В Питере семь лет простоял памятник не тому че...,Honest,Assertion
1,США официально объявили о новых санкциях проти...,Honest,Assetion
2,Facebook удалил страницу обсуждения штурма «Зо...,Honest,Assertion
3,Джонсон способен развалить Британию,Clickbait,"False actuality inference, Other inference(Sem..."
4,Спортсмен–парапланерист из Москвы погиб в гора...,Honest,Assertion
...,...,...,...
1188,«Шашлык Live» собрал аншлаг. Почему о фестивал...,Honest,"Assertion, Quote"
1189,Роналду заговорил по-русски,Clickbait,False quality inference
1190,Двое мужчин в гаражах забили насмерть человека,Honest,Assertion
1191,SSJ-100 СОВЕРШИЛ ЭКСТРЕННУЮ ПОСАДКУ В САМАРЕ И...,Honest,Assertion


In [46]:
train = train[train['Type'] != 'Dishonest']
test = test[test['Type'] != 'Dishonest']

In [47]:
cat_mapper = {'Honest': 0, 'Clickbait': 1}

In [48]:
vec = TfidfVectorizer()
vec.fit(train['Headline'].values)
vec.fit(test['Headline'].values)
X_train = vec.transform(train['Headline'].values)
X_test = vec.transform(test['Headline'].values)
y_train = train['Type'].map(cat_mapper)
y_test = test['Type'].map(cat_mapper)

In [49]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1165, 811), (128, 811), (1165,), (128,))

In [50]:
log_reg = LogisticRegression(class_weight='balanced')
log_reg.fit(X_train, y_train)
preds = log_reg.predict(X_test)
print(f1_score(preds, y_test))
print(accuracy_score(preds, y_test))
print(precision_score(preds, y_test))
print(recall_score(preds, y_test))

0.21212121212121213
0.59375
0.25925925925925924
0.1794871794871795


# Bert

In [51]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [52]:
import torch

if torch.cuda.is_available():    
    device = torch.device("cuda")

In [53]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')

0 - Honest, 1 - Dishonest

In [54]:
sentences_train = train.Headline.values
labels_train = train.Type.map(cat_mapper).values
sentences_test = train.Headline.values
labels_test = train.Type.map(cat_mapper).values

In [55]:
input_ids_train = []
input_ids_test = []

for sent in sentences_train:
    encoded_sent = tokenizer.encode(sent, add_special_tokens = True)
    input_ids_train.append(encoded_sent)

for sent in sentences_test:
    encoded_sent = tokenizer.encode(sent, add_special_tokens = True)
    input_ids_test.append(encoded_sent)

In [56]:
!pip install keras_preprocessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [57]:
from keras_preprocessing.sequence import pad_sequences

MAX_LEN = 80

input_ids_train = pad_sequences(input_ids_train, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")
input_ids_test = pad_sequences(input_ids_test, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")

In [58]:
attention_masks_train = []
attention_masks_test = []

for sent in input_ids_train:
    att_mask = [int(token_id > 0) for token_id in sent]
    attention_masks_train.append(att_mask)

for sent in input_ids_test:
    att_mask = [int(token_id > 0) for token_id in sent]
    attention_masks_test.append(att_mask)

In [59]:
train_inputs = torch.tensor(input_ids_train)
validation_inputs = torch.tensor(input_ids_test)

train_labels = torch.tensor(labels_train)
validation_labels = torch.tensor(labels_test)

train_masks = torch.tensor(attention_masks_train)
validation_masks = torch.tensor(attention_masks_test)

In [60]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [61]:
from transformers import AdamW, BertConfig, BertForSequenceClassification

In [62]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-multilingual-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [63]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [64]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 100, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [65]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(pred_flat, labels_flat)
    #return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [66]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [67]:
import random

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

loss_values = []

for epoch_i in range(0, 8):

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):

        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        #b_labels = torch.nn.functional.one_hot(b_labels)

        model.zero_grad()        

        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs[0]

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    eval_f1, eval_precision, eval_recall = 0, 0, 0
    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_f1 = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_f1 += tmp_eval_f1
        #eval_precision += tmp_eval_precision
        #eval_recall += tmp_eval_recall
        #eval_accuracy += tmp_eval_accuracy

        # Track the number of batches
        nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    #print("  accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  f1: {0:.2f}".format(eval_f1/nb_eval_steps))
    #print("  precision: {0:.2f}".format(eval_precision/nb_eval_steps))
    #print("  recall: {0:.2f}".format(eval_recall/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...

  Average training loss: 0.64
  Training epcoh took: 0:00:16

Running Validation...
  f1: 0.00
  Validation took: 0:00:05

======== Epoch 2 / 4 ========
Training...

  Average training loss: 0.54
  Training epcoh took: 0:00:16

Running Validation...
  f1: 0.00
  Validation took: 0:00:05

======== Epoch 3 / 4 ========
Training...

  Average training loss: 0.49
  Training epcoh took: 0:00:17

Running Validation...
  f1: 0.23
  Validation took: 0:00:05

======== Epoch 4 / 4 ========
Training...

  Average training loss: 0.41
  Training epcoh took: 0:00:16

Running Validation...
  f1: 0.75
  Validation took: 0:00:05

======== Epoch 5 / 4 ========
Training...

  Average training loss: 0.35
  Training epcoh took: 0:00:16

Running Validation...
  f1: 0.75
  Validation took: 0:00:05

======== Epoch 6 / 4 ========
Training...

  Average training loss: 0.35
  Training epcoh took: 0:00:16

Running Validation...
  f1: 0.75
  Validation took: 0:00:05

===

# С текстами статей

In [ ]:
import pandas as pd
df = pd.read_csv('output_text_final.csv', sep = ';')

In [ ]:
df.head()

,Unnamed: 0,Headline,Date,Source,Link,Broad Topic,Type,Manipulative Device,Html-marked Headline,Summary,Text
0,0,Разоблачитель российского допинга взялся за бр...,30.06.2018,lenta.ru,https://lenta.ru/news/2018/06/30/zeppelt/,Sports,Honest,Assertion,Разоблачитель российского допинга взялся за бр...,"Хайо Зеппельт, расследоваший применения допинг...",Немецкий журналист Хайо Зеппельт анонсировал д...
1,1,Главный тренер сборной Испании признал опаснос...,30.06.2018,lenta.ru,https://lenta.ru/news/2018/06/30/dangers/,Sports,Honest,Assertion,Главный тренер сборной Испании <trigger><ev:po...,Тренер считает Россию серьёзным соперником.,Главный тренер сборной Испании по футболу Ферн...
2,2,Дзюба признался в готовности встать на ворота ...,30.06.2018,lenta.ru,https://lenta.ru/news/2018/06/30/dzyuba/,Sports,Clickbait,False actuality inference,Дзюба <trigger><ev:neg>признался</ev:neg><trig...,"Дзюба сказал, что встанет на ворота только в с...",Нападающий сборной России Артем Дзюба не исклю...
3,3,Макаревич пригрозил силовикам адом за преследо...,30.06.2018,lenta.ru,https://lenta.ru/news/2018/06/30/hell/,"Politics, Stars",Clickbait,"False referentiality inference, Other inferenc...",Макаревич <homonymy:sem>пригрозил силовикам <e...,"Музыкант сказал: ""Ребят, ну не позорьтесь вы. ...",Лидер группы «Машина времени» Андрей Макаревич...
4,4,Власти объяснили введение платы за дождь,30.06.2018,lenta.ru,https://lenta.ru/news/2018/06/30/rain/,Urban development,Clickbait,"False actuality inference, False referentialit...",Власти <trigger>объяснили</trigger> <trigger>в...,"Власти Кунгура объяснили, что налога за дождь ...",Власти города Кунгур ответили на вопрос о введ...


In [ ]:
df[df['Type'] == 'Clickbait']

,Unnamed: 0,Headline,Date,Source,Link,Broad Topic,Type,Manipulative Device,Html-marked Headline,Summary,Text
2,2,Дзюба признался в готовности встать на ворота ...,30.06.2018,lenta.ru,https://lenta.ru/news/2018/06/30/dzyuba/,Sports,Clickbait,False actuality inference,Дзюба <trigger><ev:neg>признался</ev:neg><trig...,"Дзюба сказал, что встанет на ворота только в с...",Нападающий сборной России Артем Дзюба не исклю...
3,3,Макаревич пригрозил силовикам адом за преследо...,30.06.2018,lenta.ru,https://lenta.ru/news/2018/06/30/hell/,"Politics, Stars",Clickbait,"False referentiality inference, Other inferenc...",Макаревич <homonymy:sem>пригрозил силовикам <e...,"Музыкант сказал: ""Ребят, ну не позорьтесь вы. ...",Лидер группы «Машина времени» Андрей Макаревич...
4,4,Власти объяснили введение платы за дождь,30.06.2018,lenta.ru,https://lenta.ru/news/2018/06/30/rain/,Urban development,Clickbait,"False actuality inference, False referentialit...",Власти <trigger>объяснили</trigger> <trigger>в...,"Власти Кунгура объяснили, что налога за дождь ...",Власти города Кунгур ответили на вопрос о введ...
5,5,«Собрался весь двор и надавали тумаков» Парень...,30.06.2018,lenta.ru,https://lenta.ru/articles/2018/06/30/stas/,Incidents,Clickbait,"Other inference(Hyperbole), Other inference(Se...",«Собрался весь двор и надавали тумаков» Парень...,"Юноша с ДЦП толкнул и ударил мальчика, который...",Тысячи пользователей соцсетей поддержали 21-ле...
6,6,Штыки друзей. Почему Приднестровье не хочет от...,30.06.2018,lenta.ru,https://lenta.ru/articles/2018/06/30/transnist...,"Politics, War",Clickbait,Other inference(Not the topic of the article),Штыки друзей. <trigger><question>Почему</quest...,Правительство и жители Приднестровья выступают...,В конце июня российских военных «попросили на ...
...,...,...,...,...,...,...,...,...,...,...,...
1078,1078,"Скончался индийский йог, который не пил и не е...",27.05.2021,Ридуc/Ridus,https://www.ridus.ru/news/328227,Society,Clickbait,False quality inference,"Скончался индийский йог, <trigger>который</tri...","Умер йог, по словам которого он не ел с 12 до ...","Йог и отшельник Прахлад Джани, утверждавший, ч..."
1080,1080,Джо Байден растерял рейтинг одобрения после за...,18.08.2021,Ридуc/Ridus,https://www.ridus.ru/news/360459,Politics,Clickbait,Other inference(Hyperbole),Джо Байден <hyperbole>растерял</hyperbole> рей...,После вывода войск из Афганистана рейтинг Байд...,Сдача Афганистана боевикам исламистского движе...
1082,1082,Работавшего на Пентагон афганца могут выслать ...,07.07.2021,Ридуc/Ridus,https://www.ridus.ru/news/357667,"War, Society",Clickbait,"False referentiality inference, False cause-ef...",Работавшего на Пентагон афганца могут выслать ...,"Афганца, жителя США, могут потенциально выслат...","Гражданин Афганистана Залмай Ниязи, состоявший..."
1083,1083,"Депутат-миллионер чистосердечно признался, что...",10.08.2021,Ридуc/Ridus,https://www.ridus.ru/news/359944,Incidents,Clickbait,"False cause-effect inference, False factuality...","Депутат-миллионер чистосердечно признался, что...","Депутат охотился на медведя, сделал выстрел дл...",Депутат Законодательного собрания Камчатского ...


In [ ]:
df = df[df['Type'] != 'Dishonest']

In [ ]:
cat_mapper = {'Honest': 0, 'Clickbait': 1}

In [ ]:
df = df[pd.isna(df['Text']) == False]
headlines = df['Headline'].values
texts = df['Text'].values
labels = df['Type'].map(cat_mapper).values

In [ ]:
import torch

if torch.cuda.is_available():    
    device = torch.device("cuda")

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 75.1 MB/s eta 0:00:00


In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')

In [ ]:
input_ids = []
for ind, headline in enumerate(headlines):
    encoded_sent = tokenizer(headline, texts[ind], add_special_tokens = True, max_length = 512, truncation = True, padding = 'max_length')
    input_ids.append(encoded_sent)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [ ]:
!pip install keras_preprocessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from keras_preprocessing.sequence import pad_sequences

MAX_LEN = 512

input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")

KeyError: ignored

In [ ]:
attention_masks = []

for sent in input_ids:
    att_mask = [int(token_id > 0) for token_id in sent]
    attention_masks.append(att_mask)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
input_data = []
for ind, text in enumerate(input_ids):
    text['targets'] = labels[ind]
    input_data.append(text)

In [ ]:
train_inputs, validation_inputs = train_test_split(input_ids, test_size = 0.3)
#train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels, test_size=0.3)

In [ ]:
import torch
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)

train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)

#train_masks = torch.tensor(train_masks)
#validation_masks = torch.tensor(validation_masks)

ValueError: ignored

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 4

#train_data = TensorDataset(train_inputs, train_labels)
#train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_inputs, batch_size=batch_size)

#validation_data = TensorDataset(validation_inputs, validation_labels)
#validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_inputs, batch_size=batch_size)

In [ ]:
for batch in train_dataloader:
    print(batch)
    break

{'input_ids': [tensor([101, 101, 101, 101]), tensor([10291,   309, 39800, 47706]), tensor([92025,   330, 47233, 11637]), tensor([91202, 56138, 22503, 13646]), tensor([10185, 49460, 80863, 53464]), tensor([90137, 30965, 43039, 10392]), tensor([34797, 10353, 11863, 17940]), tensor([14597, 63541,   309,   118]), tensor([47171, 10178, 33354, 54195]), tensor([10353, 10396, 18745, 10360]), tensor([10842,   321, 45824, 93857]), tensor([16696, 51182, 12690, 32690]), tensor([54994, 11569, 11182, 10360]), tensor([75670,   309,   323, 46556]), tensor([72621, 59920, 19646,   309]), tensor([11843, 10328, 44052, 92315]), tensor([15428, 17659, 10475, 25639]), tensor([11141,   309,   102, 57247]), tensor([  309,   107, 39800, 65862]), tensor([24853,   314, 47233, 10260]), tensor([12415, 18076, 22503,   309]), tensor([  102, 45905, 80863, 44662]), tensor([33878, 10468, 43039, 11487]), tensor([64857, 74517, 11863, 90064]), tensor([  117, 12253,   309,   102]), tensor([10887,   107, 33354, 10242]), tenso

In [ ]:
from transformers import AdamW, BertConfig, BertForSequenceClassification

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-multilingual-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [ ]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 100, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

In [ ]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_f1(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    print(pred_flat)
    labels_flat = labels.flatten()
    return f1_score(pred_flat, labels_flat), precision_score(pred_flat, labels_flat), recall_score(pred_flat, labels_flat), accuracy_score(pred_flat, labels_flat)
    #return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
import random

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

loss_values = []

for epoch_i in range(0, 4):

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):

        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        b_input_ids = batch['input_ids'].to(device)
        b_input_mask = batch['attention_mask'].to(device)
        b_labels = batch['targets'].to(device)
        b_token_types = batch['token_type_ids'].to(device)
        #b_labels = torch.nn.functional.one_hot(b_labels)
        #print(b_labels)

        model.zero_grad()        

        outputs = model(b_input_ids, 
                    token_type_ids=b_token_types, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs[0]

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    eval_f1, eval_precision, eval_recall = 0, 0, 0
    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_f1, tmp_eval_precision, tmp_eval_recall, tmp_eval_accuracy = flat_f1(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_f1 += tmp_eval_f1
        eval_precision += tmp_eval_precision
        eval_recall += tmp_eval_recall
        eval_accuracy += tmp_eval_accuracy

        # Track the number of batches
        nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    print("  accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  f1: {0:.2f}".format(eval_f1/nb_eval_steps))
    print("  precision: {0:.2f}".format(eval_precision/nb_eval_steps))
    print("  recall: {0:.2f}".format(eval_recall/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")